# IIC-3800 Tópicos en CC - NLP UC

- Versiones de librerías, python 3.8.10

- numpy 1.20.3
- nltk 3.7
- gensim 4.1.2
- gcsfs 2023.3.0
- protobuf 3.20.3
- keras 2.9.0
- tensorflow 2.9.1


In [1]:
import pandas as pd

df = pd.read_csv("film_affinity.csv")
df

,critica,nota,url
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...
...,...,...,...
4795,"""Shrek"" es sin lugar a dudas una de las mejore...",9,https://www.filmaffinity.com/es/reviews/6/4945...
4796,"Muy buena e incluso diría, inteligente comedia...",8,https://www.filmaffinity.com/es/reviews/3/9420...
4797,Cuando una película consigue hacer que algo ta...,7,https://www.filmaffinity.com/es/reviews/3/9420...
4798,Una gran comedia estupida que cumple su funció...,8,https://www.filmaffinity.com/es/reviews/3/9420...


____________________________________________________________________________________________________________

## Actividad en clase

Construya un clasificador de sentimiento en **castellano** usando el dataset FilmAffinity y Spacy. Para esto haga lo siguiente:

- Cree la columna 'sentiment' a partir de **nota**. Sentiment debe considerar dos clases: **positivo** y **negativo**. Observe que nota va de 1 a 10. 
- Preprocese el texto del campo **critica** usando lo que hemos visto en clases. Note que deberá cambiar algunos procesos ya que el texto está en **castellano**.
- Particione el dataset en particiones de training/testing (90/10).
- Ajuste un modelo **word2vec** al dataset usando gensim (vea la clase de ayer).
- Cree un clasificador AWE usando estos vectores (vea la clase de hoy). 
- Evalúe el clasificador sobre la particion de test.
- Cuanto termine, me avisa para entregarle una **L (logrado)**.
- Recuerde que las L otorgan un bono en la nota final de la asignatura.


***Tiene hasta el final de la clase.***

_________________________________________________________________________________________________________________

# Solución

!python3 -m spacy download es_core_news_sm

In [2]:
import string
import re
import spacy
from spacy.lang.es.stop_words import STOP_WORDS

nlp = spacy.load("es_core_news_sm") # a spanish-based nlp model
REGX_USERNAME = r"@[A-Za-z0-9$-_@.&+]+"

def preprocessing(text):
  text = text.lower()
  text = re.sub(REGX_USERNAME, ' ', text)
  tokens = [token.text for token in nlp(text)]
  tokens = [t for t in tokens if t not in STOP_WORDS and t not in string.punctuation and len(t) > 2]
  tokens = [t for t in tokens if not t.isdigit()]

  return " ".join(tokens)


df["text_clean"] = df["critica"].apply(preprocessing)

In [3]:
df["sentiment"] = df.apply(lambda x: 0 if int(x["nota"]) < 5  else 1, axis=1)

In [4]:
df.head()

,critica,nota,url,text_clean,sentiment
0,"Bueno, bajo mi gusto, otro fracaso más de DC. ...",3,https://www.filmaffinity.com/es/reviews/1/4208...,gusto fracaso empezó año aves presa acaba año ...,0
1,Es tan terrible que podría funcionar como paro...,1,https://www.filmaffinity.com/es/reviews/1/4208...,terrible funcionar parodia sobreactuada record...,0
2,Tengo una tradición desde hace más de 5 años. ...,2,https://www.filmaffinity.com/es/reviews/1/4208...,tradición años diciembre cine blockbuster peli...,0
3,No entiendo como nadie tiene la cara de presen...,1,https://www.filmaffinity.com/es/reviews/1/4208...,entiendo cara presentar película entiendo crít...,0
4,La primera entrega de Wonder Woman (2017) no m...,4,https://www.filmaffinity.com/es/reviews/1/4208...,entrega wonder woman pareció maravilla contrar...,0


In [27]:
dataset = list(df[["text_clean", "sentiment"]].sample(frac=1).itertuples(index=False, name=None))
train_data = dataset[:4320]  # 90%
test_data = dataset[4320:] # 10%

In [28]:
X_train_text = list(list(zip(*train_data))[0])
X_test_text = list(list(zip(*test_data))[0])
y_train = list(list(zip(*train_data))[1])
y_test = list(list(zip(*test_data))[1])

In [29]:
from nltk.tokenize import RegexpTokenizer, sent_tokenize

# Initialize tokenizer
# It's also possible to try with a stemmer or to mix a stemmer and a lemmatizer
tokenizer = RegexpTokenizer('[\'a-zA-Z]+')


def tokenize(document):
    words = []

    for sentence in sent_tokenize(document):
        tokens = [t.lower() for t in tokenizer.tokenize(sentence) if len(t) > 2]
        words += tokens

    return words


In [30]:
corpus_words = []

for raw_text in X_train_text:
    words = tokenize(raw_text)
    corpus_words.append(words)


In [31]:
from gensim.models import Word2Vec

model = Word2Vec(sentences=corpus_words, vector_size=100, window=5, min_count=1, workers=4, sg=1)

In [32]:
model.wv.most_similar_cosmul(positive=['king', 'woman'], negative=['man'])

[('william', 0.99941086769104),
 ('pierre', 0.9984567165374756),
 ('eligi', 0.9984337091445923),
 ('temple', 0.9976115822792053),
 ('periodista', 0.99753737449646),
 ('devil', 0.9974064826965332),
 ('debidamente', 0.9973728060722351),
 ('tibetano', 0.9973520636558533),
 ('village', 0.9973311424255371),
 ('selznick', 0.9972866177558899)]

In [33]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

max_tokens = 50 ## Hyperparameter, input length

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_text+X_test_text)

## Vectorizing data to keep 50 words per sample.
X_train_vect = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=max_tokens, padding="post", truncating="post", value=0.)
X_test_vect  = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=max_tokens, padding="post", truncating="post", value=0.)


X_train_vect.shape, X_test_vect.shape

((4320, 50), (480, 50))

In [35]:
import numpy as np

vectors_w2v = np.asarray(model.wv.vectors)
labels_w2v = np.asarray(model.wv.index_to_key)

In [36]:
embed_len = 100

w2v_embeddings = np.zeros((len(tokenizer.index_word)+1, embed_len))

for idx, word in tokenizer.index_word.items():
    if word in labels_w2v:
        w2v_embeddings[idx] = vectors_w2v[int(np.where(labels_w2v == word)[0][0])]
    

In [60]:
import tensorflow

from keras.models import Model
from keras.layers import Dense, Embedding, Input

inputs = Input(shape=(max_tokens, ))
embeddings_layer = Embedding(input_dim=len(tokenizer.index_word)+1, output_dim=embed_len,
                             input_length=max_tokens, trainable=False, weights=[w2v_embeddings])
dense1 = Dense(128, activation="relu")
dense2 = Dense(64, activation="relu")
dense3 = Dense(2, activation="softmax")

x = embeddings_layer(inputs)
x = tensorflow.reduce_mean(x, axis=1) ### Averaged embeddings of tokens of each example
x = dense1(x)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 50)]              0         
                                                                 
 embedding_5 (Embedding)     (None, 50, 100)           5690700   
                                                                 
 tf.math.reduce_mean_3 (TFOp  (None, 100)              0         
 Lambda)                                                         
                                                                 
 dense_14 (Dense)            (None, 128)               12928     
                                                                 
 dense_15 (Dense)            (None, 64)                8256      
                                                                 
 dense_16 (Dense)            (None, 2)                 130       
                                                           

In [61]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [62]:
y_train = np.asarray(y_train)

In [63]:
model.fit(X_train_vect, y_train, batch_size=32, epochs=8)

Epoch 1/8
135/135 [==============================] - 1s 3ms/step - loss: 0.6372 - accuracy: 0.6384
Epoch 2/8
135/135 [==============================] - 0s 3ms/step - loss: 0.5761 - accuracy: 0.7097
Epoch 3/8
135/135 [==============================] - 0s 3ms/step - loss: 0.5695 - accuracy: 0.7100
Epoch 4/8
135/135 [==============================] - 0s 3ms/step - loss: 0.5633 - accuracy: 0.7146
Epoch 5/8
135/135 [==============================] - 0s 3ms/step - loss: 0.5599 - accuracy: 0.7178
Epoch 6/8
135/135 [==============================] - 0s 3ms/step - loss: 0.5546 - accuracy: 0.7188
Epoch 7/8
135/135 [==============================] - 0s 3ms/step - loss: 0.5467 - accuracy: 0.7308
Epoch 8/8
135/135 [==============================] - 0s 3ms/step - loss: 0.5424 - accuracy: 0.7273


In [64]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_test = np.asarray(y_test)
Y_preds = model.predict(X_test_vect).argmax(axis=-1)

print("Test Accuracy : {}".format(accuracy_score(y_test, Y_preds)))
print("\nClassification Report : ")
print(classification_report(y_test, Y_preds, target_names=['POS','NEG']))

15/15 [==============================] - 0s 2ms/step
Test Accuracy : 0.7395833333333334

Classification Report : 
              precision    recall  f1-score   support

         POS       0.73      0.77      0.75       240
         NEG       0.75      0.71      0.73       240

    accuracy                           0.74       480
   macro avg       0.74      0.74      0.74       480
weighted avg       0.74      0.74      0.74       480

